In [1]:
import pandas as pd
import numpy as np
from datetime import date
import requests

In [2]:
GECKO_URL = 'http://nletf-gecko-p01:8998/api/v1'
#GECKO_URL = 'http://usomm-goat-conf-p02:8998/api/v1'
ETF_PRICINGS_URL = GECKO_URL + '/etfPricings'
INDEX_PRICINGS_URL = GECKO_URL + '/indexPricings'
ALL_MAPPED_PATH = '/allMapped'
BY_RIC_PATH = '/byRic/'
TODAY =date.today()

def load_json_url(url, trade_date=TODAY):
    response = requests.get(url, params={'tradeDate': trade_date})
    response.raise_for_status()
    return response.json()
def get_data(ric,date= TODAY):
    url = "http://nletf-gecko-p01:8998/api/v1/etfPricings/byRic/" + ric + "?"
    data = load_json_url(url, trade_date=date)
    return data

In [3]:
eurostoxxEtf_df = pd.read_excel('Eurostoxx ETFs.xlsx')
etfList_dict = eurostoxxEtf_df.set_index('ProductRic')['Leverage'].to_dict()

In [4]:
def get_closePrice(output):
    closePrice = output.get('closePrice').get('closePrice')
    return closePrice

In [5]:
def get_issuerNav(output):
    x = output.get('issuerNav')
    return x

In [6]:
def get_impliedNav(output):
    x = output.get('impliedNav')
    return x


In [7]:
def get_impliedIssuerNav(output):
    x = output.get('impliedIssuerNav')
    return x


In [8]:
def get_preferredNav(output):
    x = output.get('preferredNav')
    return x

In [9]:
def get_pricingDate(output):
    x = output.get('pricingDate')
    return x

In [10]:
def get_tradeDate(output):
    x= output.get('issuerTradeDate')
    return x

In [11]:
def get_leverage(ric,etf_dict1 = etfList_dict):
    leverage = etf_dict1.get(ric)
    return leverage
    

In [12]:
def get_preferEtfPrice(ric,output):
    constituents = output.get('preferredBasketPricing').get('constituentPricings')
    pricing_date = [c.get('preferredListingPricing').get('pricingDate') for c in constituents]
    symbol = [c.get('preferredListingPricing').get('product').get('symbol') for c in constituents]
    #date = [c.get('preferredListingPricing').get('pricingDate') for c in constituents]
    preferWeight = [c.get('weight') for c in constituents]
    prefercurrency = [c.get('preferredListingPricing').get('currency') for c in constituents]
    preferAdjustedClosePrice = [c.get('preferredListingPricing').get('adjustedClosePrice') for c in constituents]
    preferContribution = [c.get('preferredListingPricing').get('contribution') for c in constituents]
    

    data = {'pricing_date': pricing_date, \
            'symbol': symbol, \
            'preferWeights': preferWeight, \
            'prefercurrency': prefercurrency, \
            'preferAdjustedClosePrice': preferAdjustedClosePrice, \
            'preferContribution': preferContribution \
           }
    preferData_df = pd.DataFrame(data)
    preferData_df['weight*adjustedPrice'] = preferData_df['preferWeights']*preferData_df['preferAdjustedClosePrice']
    #preferEtfPrice= sum(preferData_df['weight*adjustedPrice']) /get_leverage(ric) + output.get('calculationCash')
    preferEtfPrice= (sum(preferData_df['weight*adjustedPrice'])  + output.get('calculationCash'))/get_leverage(ric)
    return preferEtfPrice

In [14]:
url = "http://nletf-gecko-p01:8998/api/v1/etfPricings/byRic/" + "C50.PA" + "?"
y= load_json_url(url,trade_date = TODAY)

ConnectionError: HTTPConnectionPool(host='nletf-gecko-p01', port=8998): Max retries exceeded with url: /api/v1/etfPricings/byRic/C50.PA?tradeDate=2019-09-25 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11a95ac88>: Failed to establish a new connection: [Errno 61] Connection refused'))

Check with today data

In [ ]:
ETF_List = ['C50.PA', 'C50U.PA', 'C5L.PA', 'C5S.PA', 'CD5.PA', 'PCFD.DE', 'PCFC.DE', 'CBSTOXX50E.DE', 'CBSX5EL.DE', 
            'CBSTOXX50ER.DE', 'CBSX5TS.DE', 'E950.DE', 'XESC.DE', 'XESX.DE', 'XSSX.DE', 'ETBB.PA', 'ETDD.PA', 'ELFA.DE', 
            'ETFSTOXX50E.DE', 'H50E.L', 'CSSX5E.S', 'EUE.L', 'STX50EEX.DE', 'BSX.PA', 'BXX.PA', 'LVE.PA', 'MSE.PA',
            'MSED.PA', 'SDJE50.DE', 'SDJE5D.DE', 'FEZ.P', 'E50EUA.DE', 'VX5E.L']

In [ ]:
ETF_List_1 = ['C50.PA', 'C50U.PA', 'CD5.PA', 'CBSTOXX50E.DE', 'CBSTOXX50ER.DE', 'E950.DE', 'XESC.DE', 'XESX.DE', 
              'ETBB.PA', 'ETDD.PA', 'ELFA.DE', 'ETFSTOXX50E.DE', 'H50E.L', 'CSSX5E.S', 'EUE.L', 'STX50EEX.DE', 
              'MSE.PA', 'MSED.PA', 'SDJE50.DE', 'SDJE5D.DE', 'FEZ.P', 'E50EUA.DE', 'VX5E.L']

In [ ]:
etf_list =  []
tradeDate=  []
pricingDate=  []
issuerNav=  []
impliedNav=  []
impliedIssuerNav=  []
preferredNav=  []
preferEtfPrice =  []

for ric, leverage in etfList_dict.items():
    ric= str(ric)
    output= get_data(ric)
    etf_list.append(ric)
    tradeDate.append(get_tradeDate(output))
    pricingDate.append(get_pricingDate(output))
    issuerNav.append(get_issuerNav(output))
    impliedNav.append(get_impliedNav(output))
    impliedIssuerNav.append(get_impliedIssuerNav(output))
    preferredNav.append(get_preferredNav(output))
    preferEtfPrice.append(get_preferEtfPrice(ric,output))

In [ ]:
etf_dict = {
    'etf' : ETF_List,\
    'tradeDate' :tradeDate, \
    'pricingDate' :pricingDate, \
    'issuerNav' :issuerNav, \
    #'impliedNav' :impliedNav, \
    'impliedIssuerNav' :impliedIssuerNav, \
    'preferredNav' :preferredNav, \
    'preferEtfPrice' :preferEtfPrice \
}
etf_df = pd.DataFrame(etf_dict)

In [ ]:
etf_df['check'] = etf_df['issuerNav'] - etf_df['preferEtfPrice']
etf_df.loc[etf_df['check'] > 0.05]

In [ ]:
#Check with the data of yesterday:
def get_data_yesterday(ric,output):
    date = get_pricingDate(output)
    dataYesterday = load_json_url(url, trade_date=date)
    return dataYesterday

In [ ]:
indexClose

In [ ]:
x = get_data_yesterday('C50.PA')

In [ ]:
x